# Brain Tumor Detection with Syft & SyMPC
Oleksandr Lytvyn
dataset link: [kaggle.com/navoneel/brain-mri-images-for-brain-tumor-detection](https://www.kaggle.com/navoneel/brain-mri-images-for-brain-tumor-detection)

## Data Owner 1
#### TASKS
1. Establish the Syft local syft instance
2. Upload the data
3. Accept the request of DS to __access the data__ the data with DS
4. Retrieve the prediction results, (or allow the DS to access the prediction results)

In [1]:
import pandas as pd
import numpy as np
import syft as sy

import torch

from transformed_dataset import TransformedImageDataset

In [2]:
domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into tender_thomas... done!


In [3]:
domain_node.users.create(
    **{
        "name": "DS",
        "email": "ds@stuba.sk",
        "password": "ds_fiit",
        "budget": 100
    }
)

In [4]:
domain_node.users

,id,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,budget_spent
0,1,info@openmined.org,Jane Doe,5.55,54dd265f71c45a5e9f6fd3247a9a8ed06ed29ea5c1e5a9...,Owner,None,None,None,NaN,2022-03-09 08:12:01.421597,5.55
1,2,ds@stuba.sk,DS,100.00,2f13b3a4c89d19c664fdaec1ac6080ff609fbec72f74ba...,Data Scientist,Jane Doe,,,1.0,2022-03-09 08:13:02.640844,100.00


In [5]:
dir_path = '../../data/prepared_brain_tumor_imgs/do1'
do1_data = TransformedImageDataset(images_path=dir_path + '/imgs',
                                   labels_path=dir_path + '/labels')

print(f"do1 dataset: {len(do1_data)}")

do1 dataset: 84


In [6]:
from syft.core.adp.entity import Entity
do1_final_dataset = {}
entities = []

for idx, data in enumerate(do1_data):
    name = f"Patient {idx}"
    entity = Entity(name)
    entities.append(entity)
    image_tensor =  sy.Tensor(data[0].detach().numpy().astype(np.int32))\
        .private(min_val=0, max_val=100, entities=entity)
    do1_final_dataset[name] = image_tensor

do1_final_dataset

{'Patient 0': Tensor(child=SingleEntityPhiTensor(entity=Patient 0, child=[[[0 3 7 ... 1 1 1]
   [0 3 7 ... 1 1 1]
   [1 4 7 ... 1 1 1]
   ...
   [2 2 2 ... 2 1 1]
   [1 1 1 ... 1 1 1]
   [1 1 1 ... 1 1 1]]])),
 'Patient 1': Tensor(child=SingleEntityPhiTensor(entity=Patient 1, child=[[[2 3 4 ... 0 0 0]
   [2 1 3 ... 0 0 1]
   [1 1 3 ... 0 0 1]
   ...
   [0 0 0 ... 4 2 1]
   [2 3 2 ... 2 2 2]
   [4 4 4 ... 0 3 4]]])),
 'Patient 2': Tensor(child=SingleEntityPhiTensor(entity=Patient 2, child=[[[0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   ...
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]]])),
 'Patient 3': Tensor(child=SingleEntityPhiTensor(entity=Patient 3, child=[[[0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   ...
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]]])),
 'Patient 4': Tensor(child=SingleEntityPhiTensor(entity=Patient 4, child=[[[0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   ...
   [0 0 0 ... 0 0 

In [7]:
domain_node.load_dataset(assets=do1_final_dataset,
                         name="Data from DO1",
                         description="Brain Tumor MRIs from DO1")

Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [8]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Data from DO1,Brain Tumor MRIs from DO1,"[""Patient 0""] -> Tensor[""Patient 1""] -> Tensor[""Patient 2""] -> Tensor...",337642f9-c536-4fb2-a5f7-ebe81edbfc00


In [9]:
domain_node.requests.add_handler(action="accept")

In [22]:
domain_node.requests

""


In [18]:
published_res = domain_node.datasets[0]["Patient 0"].publish(sigma=10)

In [19]:
published_res.exists

False

In [ ]:
from syft.core.tensor.smpc.mpc_tensor import MPCTensor

mpc_data = MPCTensor()